In [1]:
import utils
import models.categorical
import math
import pickle
import numpy as np
import pandas as pd
from datetime import date

from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [2]:
fourCandleHammerParams = [
    {
        "N": [30, 40],
        "highFactor": [0.9, 0.95],
        "lowFactor": [1.05, 1.1]
    }
]

emaParams = [
    {
        "shortTermDays": [20, 40],
        "longTermDays": [50, 200]
    }
]

swingSetupParams = [
    {
        "shortTermDays": [20, 30],
        "longTermDays": [40, 60],
        "highFactor": [1.002, 1.005],
        "lowFactor": [0.995, 0.998]
    }
]

start_date = date(2010, 1, 1)
end_date = date(2021, 1, 1)
futuresList = utils.futuresList

## Hyperparameter Tuning for FourCandleHammer Strategy

In [3]:
# retrieve parameter grid
parameter_grid = list(ParameterGrid(fourCandleHammerParams))
y_var = "LONG_SHORT"
file_dir = "fourCandleHammer/perc/"

for future in tqdm(futuresList):
    print(future)

    # load data
    df = utils.prepare_data(future)
    
    # generate X vars
    X_vars = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
    
    # load X and y
    X_df, y_df = utils.generate_X_y(df, X_vars=X_vars, y_var=y_var)
    cost_df = df["CLOSE_PCT"]

    agg_results_collated = pd.DataFrame(index=list(range(len(parameter_grid))), 
                                        columns=["accuracy_SMA", "opp_cost_SMA"])
    win_results_collated = []

    # run walk forward validation 
    for i in range(len(parameter_grid)):
        param_set = parameter_grid[i]
        strategy_output = utils.fourCandleHammer(X_df['CLOSE'], param_set['N'], param_set['highFactor'], param_set['lowFactor'])
        win_results, agg_results = models.categorical.walk_forward_techIndicators(
            strategy_output = strategy_output, X = X_df, y = y_df, cost_weight = cost_df, rolling = True, 
            max_windows = 100, start_index = start_date
        )
        win_results_collated.append(win_results)
        agg_results_collated.loc[i, "accuracy_SMA"] = agg_results.loc["SMA", "accuracy"]
        agg_results_collated.loc[i, "opp_cost_SMA"] = agg_results.loc["SMA", "opp_cost"]

    # save parameters
    parameter_df = pd.DataFrame.from_records(parameter_grid)
    combined_df = pd.concat([parameter_df, agg_results_collated], axis=1)
    combined_df = combined_df.sort_values(by=["accuracy_SMA", "opp_cost_SMA"], ascending=False)
    combined_df.to_csv(f"model_metrics/categorical/{file_dir}{future}.csv", index=False)

 41%|████      | 36/88 [30:10<43:35, 50.30s/it]


ValueError: Found input variables with inconsistent numbers of samples: [531, 37]

In [ ]:
combined_df

In [ ]:
agg_results_collated

## Hyperparameter Tuning for EMA Strategy

In [ ]:
# retrieve parameter grid
parameter_grid = list(ParameterGrid(emaParams))
y_var = "LONG_SHORT"
file_dir = "ema/perc/"

for future in tqdm(futuresList):
    print(future)

    # load data
    df = utils.prepare_data(future)
    
    # generate X vars
    X_vars = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
    
    # load X and y
    X_df, y_df = utils.generate_X_y(df, X_vars=X_vars, y_var=y_var)
    cost_df = df["CLOSE_PCT"]

    agg_results_collated = pd.DataFrame(index=list(range(len(parameter_grid))), 
                                        columns=["accuracy_SMA", "opp_cost_SMA"])
    win_results_collated = []

    # run walk forward validation 
    for i in range(len(parameter_grid)):
        param_set = parameter_grid[i]
        strategy_output = utils.fourCandleHammer(X_df['CLOSE'], param_set['N'], param_set['highFactor'], param_set['lowFactor'])
        win_results, agg_results = models.categorical.walk_forward_techIndicators(
            strategy_output = strategy_output, X = X_df, y = y_df, cost_weight = cost_df, rolling = True, 
            max_windows = 100, start_index = start_date
        )
        win_results_collated.append(win_results)
        agg_results_collated.loc[i, "accuracy_SMA"] = agg_results.loc["SMA", "accuracy"]
        agg_results_collated.loc[i, "opp_cost_SMA"] = agg_results.loc["SMA", "opp_cost"]

    # save parameters
    parameter_df = pd.DataFrame.from_records(parameter_grid)
    combined_df = pd.concat([parameter_df, agg_results_collated], axis=1)
    combined_df = combined_df.sort_values(by=["accuracy_SMA", "opp_cost_SMA"], ascending=False)
    combined_df.to_csv(f"model_metrics/categorical/{file_dir}{future}.csv", index=False)